In [1]:
# Third-Party Packages
import numpy as np
import pandas as pd
import sklearn

from feature_engine.creation import CyclicalFeatures, RelativeFeatures
from feature_engine.datetime import DatetimeFeatures
from feature_engine.imputation import CategoricalImputer
from feature_engine.outliers import Winsorizer
from feature_engine.pipeline import make_pipeline
from feature_engine.selection import DropFeatures
from feature_engine.timeseries.forecasting import LagFeatures, WindowFeatures
from feature_engine.wrappers import SklearnTransformerWrapper

from holidays import country_holidays

from pandas import Categorical, DataFrame, Series
from pandas.tseries.offsets import CustomBusinessDay, DateOffset, MonthBegin, MonthEnd

from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, OrdinalEncoder

In [2]:
# Configuration for Pandas
pd.set_option("display.max_rows", 11)
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")

# Configuration for Scikit-Learn
sklearn.set_config(transform_output="pandas")

# Configuration for Demand Forecasting (Start Year, End Year, and Warmup Period (Initial Days for Demand Generation))
START, END, MONTH, WARMUP = 2023, 2024, 8, 7

In [3]:
ForwardFillTransformer = FunctionTransformer(lambda x: x.copy().ffill())
EpochTransformer = FunctionTransformer(lambda x: x.copy().apply(lambda z: z.astype("int64")), lambda x: x.copy().apply(pd.to_datetime))

## Data Ingestion and Transformation of the Nutritional Trends Index

This section focuses on the collection and preprocessing of data related to the Nutritional Trends Index, which tracks shifts in consumer preferences, dietary habits, and nutritional awareness over time. The index is sourced from consumer surveys, market research reports, and health-related databases, capturing trends such as the rise in plant-based diets, organic food consumption, and the popularity of specific nutrients or dietary supplements.

In [4]:
# Set the random seed for reproducibility.
np.random.seed(42)

# Create a data frame with a date range from START to END with a monthly frequency.
df__nti = DataFrame({"RELEASE": pd.date_range(start=f"{START}-{MONTH}", end=f"{END}-{MONTH + 2}", freq="MS")})

# Create a TARGET feature to represent the validity month (the previous month).
df__nti["TARGET"] = df__nti["RELEASE"].apply(lambda date: f"{(date - MonthBegin(1)).month_name()[:3].upper()} {(date - MonthBegin(1)).year}").astype("string")

# Add a TYPE feature to indicate that the data is actual (as opposed to forecasted) data.
df__nti["TYPE"] = Series(["ACTUAL"] * len(df__nti), dtype="string")

# Create the NTI by applying a sinusoidal function to the index. The VALUE column is computed as a combination of a 
# sine wave where 5 determines the height of the wave and 13 defines the frequency of the oscillations, a linear trend 
# where 0.5 controls the slope.
df__nti["VALUE"] = (5 * np.sin(2 * np.pi * df__nti.index / 4)) + (1 * df__nti.index)

# Introduce some outliers to the NTI to simulate spikes due to events like a popular documentary or media coverage.
df__nti.loc[[11, 5], "VALUE"] += np.random.normal(30, 30, size=2)  

# Scale the NTI to start at a value of 120 and round the result to whole numbers.
df__nti["VALUE"] = (120 + (df__nti["VALUE"] - df__nti["VALUE"][0])).round(0).astype("Float64")

# Define the validity period for each data point, covering the month prior to the release date.
df__nti["VALID_FROM"] = df__nti["RELEASE"] - DateOffset(months=1)
df__nti["VALID_TO"] = df__nti["VALID_FROM"] + pd.offsets.MonthEnd()

# Reset the index again to ensure it is clean and sequential after all operations and export the processed data to a CSV file.
df__nti.reset_index(drop=True, inplace=True)
df__nti.to_csv("../datasets/nutritional-trends-index.csv", index=False)

In [5]:
df__nti

,RELEASE,TARGET,TYPE,VALUE,VALID_FROM,VALID_TO
0,2023-08-01,JUL 2023,ACTUAL,120.0000,2023-07-01,2023-07-31
1,2023-09-01,AUG 2023,ACTUAL,126.0000,2023-08-01,2023-08-31
2,2023-10-01,SEP 2023,ACTUAL,122.0000,2023-09-01,2023-09-30
3,2023-11-01,OCT 2023,ACTUAL,118.0000,2023-10-01,2023-10-31
4,2023-12-01,NOV 2023,ACTUAL,124.0000,2023-11-01,2023-11-30
...,...,...,...,...,...,...
10,2024-06-01,MAY 2024,ACTUAL,130.0000,2024-05-01,2024-05-31
11,2024-07-01,JUN 2024,ACTUAL,171.0000,2024-06-01,2024-06-30
12,2024-08-01,JUL 2024,ACTUAL,132.0000,2024-07-01,2024-07-31
13,2024-09-01,AUG 2024,ACTUAL,138.0000,2024-08-01,2024-08-31


In [6]:
df__nti.dtypes.to_frame().transpose()

,RELEASE,TARGET,TYPE,VALUE,VALID_FROM,VALID_TO
0,datetime64[ns],string[python],string[python],Float64,datetime64[ns],datetime64[ns]


## Data Ingestion and Transformation of the Consumer Climate Index

This section focuses on the collection and preprocessing of the Consumer Climate Index data. It involves importing the data from external sources, cleaning it, and transforming it into a format suitable for analysis. The Consumer Climate Index, published by Gesellschaft für Konsumforschung, is a crucial economic indicator that measures the sentiment and confidence of German consumers regarding their economic outlook, personal financial situation, and willingness to spend. This index is widely used to assess consumer behavior and predict future economic trends in Germany. The data for this analysis has been downloaded manually from the [Investing.com Economic Calendar](https://www.investing.com/economic-calendar/gfk-german-consumer-climate-359), which allows access to both historical and forecast data.

In [7]:
df__cci = pd.read_csv("../datasets/consumer-climate-index-raw.tsv", sep="\t", usecols=[0, 2, 4], names=["RELEASE", "FORECAST", "ACTUAL"], header=0)
df__cci["RELEASE"] = df__cci["RELEASE"].str.split(" (", regex=False, expand=True).drop(1, axis=1).apply(pd.to_datetime)
df__cci = pd.melt(df__cci, id_vars="RELEASE", value_vars=["ACTUAL", "FORECAST"], var_name="TYPE", value_name="VALUE").sort_values(["RELEASE", "TYPE"], ascending=[False, False]).reset_index(drop=True)
df__cci["VALID_FROM"] = Series(np.where(df__cci["TYPE"] == "ACTUAL", df__cci["RELEASE"], pd.NA)).apply(pd.to_datetime)
df__cci["VALID_TO"] = df__cci["VALID_FROM"].apply(lambda x: x + MonthEnd(0))
df__cci["VALID_FROM"] = np.where(df__cci["TYPE"] == "FORECAST", df__cci["RELEASE"].apply(lambda x: x + MonthBegin(0)), df__cci["VALID_FROM"])
df__cci["VALID_TO"] = df__cci["VALID_TO"].fillna(df__cci["VALID_FROM"].shift(1).apply(lambda x: x - DateOffset(days=1))).fillna(df__cci["VALID_FROM"].apply(lambda x: x + MonthEnd()))
df__cci["TYPE"] = df__cci["TYPE"].astype("string")
df__cci["VALUE"] = df__cci["VALUE"].astype("Float64")
df__cci.insert(1, "TARGET", df__cci["VALID_FROM"].apply(lambda date: f"{date.month_name()[:3].upper()} {date.year}").astype("string"))
df__cci.reset_index(drop=True, inplace=True)
df__cci.to_csv("../datasets/consumer-climate-index.csv", index=False)

In [8]:
df__cci

,RELEASE,TARGET,TYPE,VALUE,VALID_FROM,VALID_TO
0,2024-08-27,SEP 2024,FORECAST,-22.0000,2024-09-01,2024-09-30
1,2024-08-27,AUG 2024,ACTUAL,-18.6000,2024-08-27,2024-08-31
2,2024-07-24,AUG 2024,FORECAST,-18.4000,2024-08-01,2024-08-26
3,2024-07-24,JUL 2024,ACTUAL,-21.6000,2024-07-24,2024-07-31
4,2024-06-26,JUL 2024,FORECAST,-21.8000,2024-07-01,2024-07-23
...,...,...,...,...,...,...
53,2022-06-28,JUN 2022,ACTUAL,-26.2000,2022-06-28,2022-06-30
54,2022-05-25,JUN 2022,FORECAST,-26.0000,2022-06-01,2022-06-27
55,2022-05-25,MAY 2022,ACTUAL,-26.6000,2022-05-25,2022-05-31
56,2022-04-27,MAY 2022,FORECAST,-26.5000,2022-05-01,2022-05-24


In [9]:
df__cci.dtypes.to_frame().transpose()

,RELEASE,TARGET,TYPE,VALUE,VALID_FROM,VALID_TO
0,datetime64[ns],string[python],string[python],Float64,datetime64[ns],datetime64[ns]


## Data Ingestion and Transformation of German Public Holidays for North Rhine-Westphalia

This section deals with the acquisition and preprocessing of data regarding public holidays in North Rhine-Westphalia, Germany. The data is cleaned and structured to align with the temporal scope of the analysis.

In [10]:
df__holidays = DataFrame(country_holidays(country="DE", subdiv="NW", years=list(range(START, END + 1))).items(), columns=["DATE", "HOLIDAY"])
df__holidays["DATE"] = df__holidays["DATE"].apply(pd.to_datetime)
df__holidays = df__holidays.astype({"HOLIDAY": "string"})

In [11]:
df__holidays

,DATE,HOLIDAY
0,2024-01-01,Neujahr
1,2024-03-29,Karfreitag
2,2024-04-01,Ostermontag
3,2024-05-01,Erster Mai
4,2024-05-09,Christi Himmelfahrt
...,...,...
17,2023-10-03,Tag der Deutschen Einheit
18,2023-12-25,Erster Weihnachtstag
19,2023-12-26,Zweiter Weihnachtstag
20,2023-06-08,Fronleichnam


In [12]:
df__holidays.dtypes.to_frame().transpose()

,DATE,HOLIDAY
0,datetime64[ns],string[python]


## Synthetic Data Generation for Single-Product Demand Forecasting

This section is dedicated to creating synthetic data for forecasting demand for a single product. The process includes simulating realistic demand scenarios based on predefined parameters.

### Data Simulation

This subsection details the process of generating synthetic data, including the methods and parameters used to simulate demand patterns for the product.

In [13]:
# Set a random seed for reproducibility.
np.random.seed(1207)

# Create a custom business day frequency that includes Monday to Saturday and excludes holidays.
bday = CustomBusinessDay(weekmask="Mon Tue Wed Thu Fri Sat", holidays=df__holidays["DATE"].tolist())

# Create a data frame with a date range from 2023-08-31 to 2024-08-30, using the custom business day frequency.
df__demand__raw = DataFrame(data={"ODATE": pd.date_range(start=f"{START}-{MONTH}-31", end=f"{END}-{MONTH}-30", freq=bday)})
df__demand__raw["ODATEINT"] = df__demand__raw["ODATE"].astype("int64")

# Calculate the target date by adding one custom business day to each prediction date.
df__demand__raw["TDATE"] = df__demand__raw["ODATE"].apply(lambda date: date + bday)
df__demand__raw["TDATEINT"] = df__demand__raw["TDATE"].astype("int64")

# Categorize the target date's month into seasons, one-hot encode the SEASON feature, and merge it into the data frame.
df__demand__raw["SEASON"] = pd.cut(df__demand__raw["TDATE"].dt.month, bins=[-999, 2, 5, 8, 11, 999], labels=["WINTER", "SPRING", "SUMMER", "FALL", "WINTER"], ordered=False)
df__demand__raw = df__demand__raw.merge(pd.get_dummies(df__demand__raw["SEASON"], prefix="OHE__SEASON", dtype="Int64"), left_index=True, right_index=True, how="left")

# Create a DOW feature representing the day of the week (abbreviated to 3 letters and in uppercase), then merge it into the data frame.
df__demand__raw["DOW"] = Categorical(df__demand__raw["TDATE"].dt.day_name().str[:3].str.upper(), categories=["MON", "TUE", "WED", "THU", "FRI", "SAT"])
df__demand__raw = df__demand__raw.merge(pd.get_dummies(df__demand__raw["DOW"], prefix="OHE__DOW", dtype="Int64"), left_index=True, right_index=True, how="left")

# Create a feature for the day of the month (DOM), then apply cyclical encoding using sine and cosine transformations.
df__demand__raw["DOM"] = df__demand__raw["TDATE"].dt.day.astype("Int64")
df__demand__raw["CE__DOM_SIN"] = np.sin((df__demand__raw["DOM"]) / 31 * 2 * np.pi)
df__demand__raw["CE__DOM_COS"] = np.cos((df__demand__raw["DOM"]) / 31 * 2 * np.pi)

# Count holidays between prediction and target dates for each row, shift by -1 to align, and fill missing values with 0.
df__demand__raw["HOLIDAYS"] = df__demand__raw.apply(lambda df: len(df__holidays[(df["ODATE"] < df__holidays["DATE"]) & (df__holidays["DATE"] < df["TDATE"])]), axis=1).shift(-1, fill_value=0).astype("Int64")

# Generate base demand store them in the DEMAND column.
df__demand__raw["DEMAND"] = Series([500] * len(df__demand__raw), dtype="Int64")

# Create a MARKETING feature with levels (LOW, MEDIUM, HIGH), convert to ordinal codes (LOW=0, MEDIUM=1, HIGH=2), with probabilities 65%, 25%, 10%.
df__demand__raw["MARKETING"] = Categorical(np.random.choice(["LOW", "MEDIUM", "HIGH"], size=len(df__demand__raw), replace=True, p=[0.65, 0.25, 0.10]), categories=["LOW", "MEDIUM", "HIGH"])
df__demand__raw["ORD__MARKETING"] = df__demand__raw["MARKETING"].cat.codes.astype("Int64")

# Create a PROMOTION feature with levels (NONE, DISCOUNT, BOGO), one-hot encode it, and merge back into the data frame. Probability: 75%, 12.5%, 12.5%.
df__demand__raw["PROMOTION"] = Categorical(np.random.choice(["NONE", "DISCOUNT", "BOGO"], size=len(df__demand__raw), replace=True, p=[0.75, 0.125, 0.125]), categories=["NONE", "DISCOUNT", "BOGO"])
df__demand__raw = df__demand__raw.merge(pd.get_dummies(df__demand__raw["PROMOTION"], prefix="OHE__PROMOTION", dtype="Int64"), left_index=True, right_index=True, how="left")

# Generate artificially missing values for the PROMOTION feature.
df__demand__raw["IMP__PROMOTION"] = df__demand__raw["PROMOTION"]
df__demand__raw.loc[df__demand__raw[df__demand__raw["IMP__PROMOTION"] == "NONE"].sample(7).index, "PROMOTION"] = pd.NA

# Set Competitor Activity Flag (CAF) for randomly selected 4 weeks with exactly 6 entries.
df__demand__raw["CAF"] = Categorical(["NO"] * len(df__demand__raw), categories=["NO", "YES"], ordered=False)
weeks = df__demand__raw.groupby(lambda x: f"{df__demand__raw.loc[x, "TDATE"].year}-{df__demand__raw.loc[x, "TDATE"].isocalendar().week}").apply(lambda group: group.index.tolist(), include_groups=True).reset_index(name="indices", drop=True)
df__demand__raw.loc[weeks[weeks.map(len) == 6].sample(4, random_state=1207).explode().tolist(), "CAF"] = "YES"
df__demand__raw["ORD__CAF"] = df__demand__raw["CAF"].cat.codes.astype("Int64")

# Map the consumer climate index to each target date based on the forecast period.
df__demand__raw = df__demand__raw.merge(df__cci[["VALID_FROM", "VALID_TO", "TYPE", "VALUE"]], how="cross")
df__demand__raw["MERGE_FROM"] = np.where(df__demand__raw["TYPE"] == "ACTUAL", df__demand__raw["VALID_FROM"] + DateOffset(1), df__demand__raw["VALID_FROM"])
df__demand__raw["MERGE_TO"] = np.where(df__demand__raw["TYPE"] == "FORECAST", df__demand__raw["VALID_TO"] + DateOffset(1), df__demand__raw["VALID_TO"])
df__demand__raw = df__demand__raw[(df__demand__raw["TDATE"] >= df__demand__raw["MERGE_FROM"]) & (df__demand__raw["TDATE"] <= df__demand__raw["MERGE_TO"])]
df__demand__raw = df__demand__raw.drop(["VALID_FROM", "VALID_TO", "TYPE", "MERGE_FROM", "MERGE_TO"], axis=1).reset_index(drop=True)
df__demand__raw = df__demand__raw.rename({"VALUE": "CCI"}, axis=1)

# Map the nutritional trends index to each prediction date and a winsorized version of the feature.
df__demand__raw = df__demand__raw.merge(df__nti[["RELEASE", "VALUE"]], how="cross")
df__demand__raw["MERGE_FROM"] = df__demand__raw["RELEASE"] + DateOffset(days=1)
df__demand__raw["MERGE_TO"] = df__demand__raw["RELEASE"] + DateOffset(months=1)
df__demand__raw = df__demand__raw[(df__demand__raw["TDATE"] >= df__demand__raw["MERGE_FROM"]) & (df__demand__raw["TDATE"] <= df__demand__raw["MERGE_TO"])]
df__demand__raw = df__demand__raw.drop(["RELEASE", "MERGE_FROM", "MERGE_TO"], axis=1).reset_index(drop=True)
df__demand__raw = df__demand__raw.rename({"VALUE": "NTI"}, axis=1)
df__demand__raw["WIN__NTI"] = Winsorizer(capping_method="iqr", tail="right", fold=1.5, variables=["NTI"]).fit_transform(df__demand__raw)["NTI"]

# Simulate temperature as a cosine function of the day of the year, varying between 10 - 35°C, peaking around day 200.
df__demand__raw["TEMPERATURE"] = ((25 * np.cos(((2 * np.pi) / 365) * (df__demand__raw["TDATE"].dt.day_of_year - 200)) + 10) + np.random.normal(0, 1, len(df__demand__raw))).round(1).astype("Float64")

# Generate artificially missing values for the TEMPERATURE feature.
df__demand__raw["IMP__TEMPERATURE"] = df__demand__raw["TEMPERATURE"]
df__demand__raw.loc[df__demand__raw.sample(8).index, "TEMPERATURE"] = pd.NA
#df__demand__raw["IMP__TEMPERATURE"] = KNNImputer(n_neighbors=4, weights="distance").fit_transform(df__demand__raw[["TDATEINT", "TEMPERATURE"]])["TEMPERATURE"].astype("Float64")
df__demand__raw["IMP__TEMPERATURE"] = ForwardFillTransformer.fit_transform(df__demand__raw[["TEMPERATURE"]])["TEMPERATURE"].astype("Float64")

# Simulate precipitation as a cosine function of the day, varying between 0 - 0.5 units, peaking around day 15.
df__demand__raw["PRECIPITATION"] = (0.25 * (1 + np.cos((2 * np.pi * (df__demand__raw["TDATE"].dt.day_of_year - 15)) / 365)) + 0).round(4).astype("Float64")

# Calculate the pruchase price with a base price, seasonal adjustments, and noise.
df__demand__raw["PPRC"] = (2.99 + 1.5 * df__demand__raw["OHE__SEASON_WINTER"] - 1 * df__demand__raw["OHE__SEASON_SUMMER"] + np.divide(np.random.randint(low=-25, high=26, size=len(df__demand__raw)), 100)).round(2)

# Compute the retail price by applying markups and rounding to end in 9.
df__demand__raw["RPRC"] = 1.20 * df__demand__raw["PPRC"] * df__demand__raw["OHE__PROMOTION_DISCOUNT"] + 1.25 * df__demand__raw["PPRC"] * df__demand__raw["OHE__PROMOTION_NONE"] + 1.25 * df__demand__raw["PPRC"] * df__demand__raw["OHE__PROMOTION_BOGO"]
df__demand__raw["RPRC"] = np.divide(np.ceil((df__demand__raw["RPRC"] * 100)) + (9 - np.ceil((df__demand__raw["RPRC"] * 100)) % 10), 100).round(2).astype("Float64")

# Add interaction term for competitor and marketing activities.
df__demand__raw["INT__CAF_MARKETING"] = df__demand__raw["ORD__CAF"] * df__demand__raw["ORD__MARKETING"]

# Add interaction term for marketing activity and promotion.
df__demand__raw["INT__MARKETING_PROMOTION_NONE"] = df__demand__raw["ORD__MARKETING"] * df__demand__raw["OHE__PROMOTION_NONE"]
df__demand__raw["INT__MARKETING_PROMOTION_DISCOUNT"] = df__demand__raw["ORD__MARKETING"] * df__demand__raw["OHE__PROMOTION_DISCOUNT"]
df__demand__raw["INT__MARKETING_PROMOTION_BOGO"] = df__demand__raw["ORD__MARKETING"] * df__demand__raw["OHE__PROMOTION_BOGO"]

# Generate random noise to simulate variability in demand.
df__demand__raw["NOISE"] = Series(np.random.randint(low=-20, high=21, size=len(df__demand__raw))).astype("Float64")

# Add contributions from seasonal effects to DEMAND value.
df__demand__raw["DEMAND"] += -1 *  10 * df__demand__raw["OHE__SEASON_FALL"]
df__demand__raw["DEMAND"] += -1 *  20 * df__demand__raw["OHE__SEASON_WINTER"]
df__demand__raw["DEMAND"] += -1 *   5 * df__demand__raw["OHE__SEASON_SPRING"]
df__demand__raw["DEMAND"] +=  1 *   0 * df__demand__raw["OHE__SEASON_SUMMER"]

# Add contributions from day-of-week effects to DEMAND value.
df__demand__raw["DEMAND"] +=  1 *   0 * df__demand__raw["OHE__DOW_MON"]
df__demand__raw["DEMAND"] +=  1 *  30 * df__demand__raw["OHE__DOW_TUE"]
df__demand__raw["DEMAND"] += -1 *  10 * df__demand__raw["OHE__DOW_WED"]
df__demand__raw["DEMAND"] +=  1 *  20 * df__demand__raw["OHE__DOW_THU"]
df__demand__raw["DEMAND"] +=  1 *  50 * df__demand__raw["OHE__DOW_FRI"]
df__demand__raw["DEMAND"] +=  1 *  80 * df__demand__raw["OHE__DOW_SAT"]

# Add contributions from the cyclical encoding of the day of the month to DEMAND value.
df__demand__raw["DEMAND"] +=  1 *  10 * df__demand__raw["CE__DOM_SIN"]
df__demand__raw["DEMAND"] +=  1 *  10 * df__demand__raw["CE__DOM_COS"]

# Add contribution from holidays to DEMAND value.
df__demand__raw["DEMAND"] +=  1 *  50 * df__demand__raw["HOLIDAYS"]

# Add contributions from promotion and marketing effects to DEMAND value.
df__demand__raw["DEMAND"] +=  1 *  10 * df__demand__raw["ORD__MARKETING"]
df__demand__raw["DEMAND"] +=  1 *   0 * df__demand__raw["OHE__PROMOTION_NONE"]
df__demand__raw["DEMAND"] +=  1 *  25 * df__demand__raw["OHE__PROMOTION_DISCOUNT"]
df__demand__raw["DEMAND"] +=  1 *  10 * df__demand__raw["OHE__PROMOTION_BOGO"]
df__demand__raw["DEMAND"] += -1 *   5 * df__demand__raw["ORD__CAF"]

# Add contribution from ecomomic effects to DEMAND value.
df__demand__raw["DEMAND"] +=  1 *  10 * df__demand__raw["CCI"]
df__demand__raw["DEMAND"] +=  1 * 0.1 * df__demand__raw["WIN__NTI"]

# Add contributions from weather effects to DEMAND value.
df__demand__raw["DEMAND"] += -1 *   1 * df__demand__raw["IMP__TEMPERATURE"]
# df__demand__raw["DEMAND"] += 0.1 * df__demand__raw["IMP__TEMPERATURE"] +  (-0.1 * ((df__demand__raw["IMP__TEMPERATURE"] - 20) ** 2)) + 200
df__demand__raw["DEMAND"] += -1 *  10 * df__demand__raw["PRECIPITATION"]

# Add contributions from price effects to DEMAND value.
df__demand__raw["DEMAND"] += -1 *  20 * df__demand__raw["RPRC"]

# Add contributions from interaction effects to DEMAND value.
df__demand__raw["DEMAND"] +=  1 *   4 * df__demand__raw["INT__CAF_MARKETING"]
df__demand__raw["DEMAND"] +=  1 *   0 * df__demand__raw["INT__MARKETING_PROMOTION_NONE"]
df__demand__raw["DEMAND"] +=  1 *   4 * df__demand__raw["INT__MARKETING_PROMOTION_DISCOUNT"]
df__demand__raw["DEMAND"] +=  1 *  16 * df__demand__raw["INT__MARKETING_PROMOTION_BOGO"]

# Add contributions from the gaussian noise to DEMAND value.
df__demand__raw["DEMAND"] +=  1 *   1 * df__demand__raw["NOISE"]

# Adjust the noise to ensure the DEMAND value is an integer.
df__demand__raw["NOISE"] -= df__demand__raw["DEMAND"] - df__demand__raw["DEMAND"].round(0)
df__demand__raw["DEMAND"] = df__demand__raw["DEMAND"].round(0)

# Create lagged demand features (1, 2 and 3 business days before) by shifting the DEMAND column.
df__demand__raw.insert(21, "LAG__DEMAND_1BD", df__demand__raw.loc[0:7, "DEMAND"].shift(1).astype("Int64"))
df__demand__raw.insert(22, "LAG__DEMAND_2BD",df__demand__raw.loc[0:8, "DEMAND"].shift(2).astype("Int64"))
df__demand__raw.insert(23, "LAG__DEMAND_3BD", df__demand__raw.loc[0:9, "DEMAND"].shift(3).astype("Int64"))

# Calculate the rolling mean and standard deviation of demand over the warmup period.
df__demand__raw.insert(24, f"LAG__DEMAND_{WARMUP}BD_MEAN", df__demand__raw["LAG__DEMAND_1BD"].rolling(WARMUP).mean().astype("Float64"))
df__demand__raw.insert(25, f"LAG__DEMAND_{WARMUP}BD_STD", df__demand__raw["LAG__DEMAND_1BD"].rolling(WARMUP).std().astype("Float64"))

# Simulate demand based on all dynamic features.
for i in range(WARMUP, len(df__demand__raw)):

    # Add contributions from past demand values and their statistics to DEMAND value.
    df__demand__raw.loc[i:i, "DEMAND"] += 1 * 0.0500 * df__demand__raw.loc[i:i, "LAG__DEMAND_1BD"].item()
    df__demand__raw.loc[i:i, "DEMAND"] += 1 * 0.0250 * df__demand__raw.loc[i:i, "LAG__DEMAND_2BD"].item()
    df__demand__raw.loc[i:i, "DEMAND"] += 1 * 0.0125 * df__demand__raw.loc[i:i, "LAG__DEMAND_3BD"].item()
    df__demand__raw.loc[i:i, "DEMAND"] += 1 * 0.0100 * df__demand__raw.loc[i:i, f"LAG__DEMAND_{WARMUP}BD_MEAN"].item()
    df__demand__raw.loc[i:i, "DEMAND"] += 1 * 0.0050 * df__demand__raw.loc[i:i, f"LAG__DEMAND_{WARMUP}BD_STD"].item()

    # Adjust the noise to ensure the DEMAND value is an integer.
    df__demand__raw.loc[i:i, "NOISE"] -= df__demand__raw.loc[i:i, "DEMAND"].item() - df__demand__raw.loc[i:i, "DEMAND"].round(0).item()
    df__demand__raw.loc[i:i, "DEMAND"] = df__demand__raw.loc[i:i, "DEMAND"].round(0).item()

    # Update the DEMAND and lagged demand features for the subsequent rows based on the current TARGET value.
    df__demand__raw.loc[i+1:i+1, "LAG__DEMAND_1BD"] = df__demand__raw.loc[i:i, "DEMAND"].item()
    df__demand__raw.loc[i+2:i+2, "LAG__DEMAND_2BD"] = df__demand__raw.loc[i:i, "DEMAND"].item()
    df__demand__raw.loc[i+3:i+3, "LAG__DEMAND_3BD"] = df__demand__raw.loc[i:i, "DEMAND"].item()

    # Recalculate the rolling mean and standard deviation of demand over the warmup period.
    df__demand__raw[f"LAG__DEMAND_{WARMUP}BD_MEAN"] = df__demand__raw["LAG__DEMAND_1BD"].rolling(WARMUP).mean().astype("Float64")
    df__demand__raw[f"LAG__DEMAND_{WARMUP}BD_STD"] = df__demand__raw["LAG__DEMAND_1BD"].rolling(WARMUP).std().astype("Float64")

# Calculate and merge demand from 7 days before the target date into the current data frame.
df__demand__raw["DATE_M7CD"] = df__demand__raw["TDATE"].apply(lambda date: date - DateOffset(days=7))
df__demand__raw.insert(26, "DEMAND7CD", df__demand__raw.merge(df__demand__raw[["TDATE", "DEMAND"]].rename({"TDATE": "DATE_M7CD", "DEMAND": "DEMAND7CD"}, axis=1), on="DATE_M7CD", how="left")["DEMAND7CD"])

# Cleanup and Finalization
df__demand__raw["DEMAND7CD"] = df__demand__raw["DEMAND7CD"].ffill()
df__demand__raw.loc[0:WARMUP - 2, "DEMAND7CD"] = np.random.randint(low=200, high=299, size=WARMUP - 1)
df__demand__raw[["DEMAND", "DEMAND7CD"]] = df__demand__raw[["DEMAND", "DEMAND7CD"]].astype("Int64")
df__demand__raw = df__demand__raw.drop("DATE_M7CD", axis=1)
df__demand__raw = df__demand__raw.reset_index(drop=True)

# Calculate the buyer's order quantity as demand from 7 days ago plus a 10% buffer.
df__demand__raw["PQTY"] = (df__demand__raw["DEMAND7CD"] * 1.1).round().astype("Int64")

# Compute the number of products sold.
df__demand__raw["SQTY"] = df__demand__raw["DEMAND"].case_when([(lambda demand: demand >= df__demand__raw["PQTY"], df__demand__raw["PQTY"]), (lambda demand: demand < df__demand__raw["PQTY"], df__demand__raw["DEMAND"])])

# Compute inventory levels by subtracting current demand from order quantity.
df__demand__raw["INVENTORY"] = df__demand__raw["PQTY"] - df__demand__raw["DEMAND"]

# Calculate gross profit per unit as the difference between retail price and purchase price.
df__demand__raw["UGP"] = (df__demand__raw["RPRC"] - df__demand__raw["PPRC"]).round(2)

# Calculate sales revenue (SR) and purchase cost (PC).
df__demand__raw["SR"] = (df__demand__raw["RPRC"] * df__demand__raw["SQTY"]).round(2)
df__demand__raw["PC"] = (df__demand__raw["PPRC"] * df__demand__raw["PQTY"]).round(2)

# Calculate gross profit as the difference between sales revenue and purchase cost.
df__demand__raw["GP"] = (df__demand__raw["SR"] - df__demand__raw["PC"]).round(2)

# Calculate cost of excessive inventory (CEI) and cost of lost sales (CLS).
df__demand__raw["CEI"] = df__demand__raw["INVENTORY"].case_when([(lambda x: x > 0, df__demand__raw["INVENTORY"] * df__demand__raw["PPRC"]), (lambda x: x <= 0, 0)]).round(2)
df__demand__raw["CLS"] = df__demand__raw["INVENTORY"].case_when([(lambda x: x < 0,  -1 * df__demand__raw["INVENTORY"] * df__demand__raw["UGP"]), (lambda x: x >= 0, 0)]).round(2)

# Calculate net profit as gross profit minus cost of lost sales.
df__demand__raw["NP"] = (df__demand__raw["GP"] - df__demand__raw["CLS"]).round(2)


In [14]:
df__demand__raw

,ODATE,ODATEINT,TDATE,TDATEINT,SEASON,OHE__SEASON_FALL,OHE__SEASON_SPRING,OHE__SEASON_SUMMER,OHE__SEASON_WINTER,DOW,OHE__DOW_MON,OHE__DOW_TUE,OHE__DOW_WED,OHE__DOW_THU,OHE__DOW_FRI,OHE__DOW_SAT,DOM,CE__DOM_SIN,CE__DOM_COS,HOLIDAYS,DEMAND,LAG__DEMAND_1BD,LAG__DEMAND_2BD,LAG__DEMAND_3BD,LAG__DEMAND_7BD_MEAN,LAG__DEMAND_7BD_STD,DEMAND7CD,MARKETING,ORD__MARKETING,PROMOTION,OHE__PROMOTION_NONE,OHE__PROMOTION_DISCOUNT,OHE__PROMOTION_BOGO,IMP__PROMOTION,CAF,ORD__CAF,CCI,NTI,WIN__NTI,TEMPERATURE,IMP__TEMPERATURE,PRECIPITATION,PPRC,RPRC,INT__CAF_MARKETING,INT__MARKETING_PROMOTION_NONE,INT__MARKETING_PROMOTION_DISCOUNT,INT__MARKETING_PROMOTION_BOGO,NOISE,PQTY,SQTY,INVENTORY,UGP,SR,PC,GP,CEI,CLS,NP
0,2023-08-31,1693440000000000000,2023-09-01,1693526400000000000,FALL,1,0,0,0,FRI,0,0,0,0,1,0,1,0.2013,0.9795,0,218,<NA>,<NA>,<NA>,<NA>,<NA>,276,MEDIUM,1,NONE,1,0,0,NONE,NO,0,-25.5000,120.0000,120.0000,28.1000,28.1000,0.0759,3.1800,3.9900,0,1,0,0,7.8507,304,218,86,0.8100,869.8200,966.7200,-96.9000,273.4800,0.0000,-96.9000
1,2023-09-01,1693526400000000000,2023-09-02,1693612800000000000,FALL,1,0,0,0,SAT,0,0,0,0,0,1,2,0.3944,0.9190,0,240,218,<NA>,<NA>,<NA>,<NA>,261,MEDIUM,1,NONE,1,0,0,NONE,NO,0,-25.5000,126.0000,126.0000,28.0000,28.0000,0.0790,3.1200,3.9900,0,1,0,0,-2.1431,287,240,47,0.8700,957.6000,895.4400,62.1600,146.6400,0.0000,62.1600
2,2023-09-02,1693612800000000000,2023-09-04,1693785600000000000,FALL,1,0,0,0,MON,1,0,0,0,0,0,4,0.7248,0.6890,0,177,240,218,<NA>,<NA>,<NA>,284,LOW,0,DISCOUNT,0,1,0,DISCOUNT,YES,1,-25.5000,126.0000,126.0000,26.7000,26.7000,0.0854,3.1800,3.8900,0,0,0,0,0.6164,312,177,135,0.7100,688.5300,992.1600,-303.6300,429.3000,0.0000,-303.6300
3,2023-09-04,1693785600000000000,2023-09-05,1693872000000000000,FALL,1,0,0,0,TUE,0,1,0,0,0,0,5,0.8486,0.5290,0,199,177,240,218,<NA>,<NA>,236,LOW,0,DISCOUNT,0,1,0,DISCOUNT,YES,1,-25.5000,126.0000,126.0000,<NA>,26.7000,0.0887,2.8000,3.3900,0,0,0,0,-16.9891,260,199,61,0.5900,674.6100,728.0000,-53.3900,170.8000,0.0000,-53.3900
4,2023-09-05,1693872000000000000,2023-09-06,1693958400000000000,FALL,1,0,0,0,WED,0,0,1,0,0,0,6,0.9378,0.3473,0,154,199,177,240,<NA>,<NA>,264,LOW,0,NONE,1,0,0,NONE,YES,1,-25.5000,126.0000,126.0000,25.8000,25.8000,0.0920,3.1800,3.9900,0,0,0,0,15.0694,290,154,136,0.8100,614.4600,922.2000,-307.7400,432.4800,0.0000,-307.7400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,2024-08-26,1724630400000000000,2024-08-27,1724716800000000000,SUMMER,0,0,1,0,TUE,0,1,0,0,0,0,27,-0.7248,0.6890,0,314,307,372,344,311.4286,36.5598,318,MEDIUM,1,NONE,1,0,0,NONE,YES,1,-18.4000,132.0000,132.0000,27.7000,27.7000,0.0640,2.0000,2.5900,1,1,0,0,-5.9488,350,314,36,0.5900,813.2600,700.0000,113.2600,72.0000,0.0000,113.2600
299,2024-08-27,1724716800000000000,2024-08-28,1724803200000000000,SUMMER,0,0,1,0,WED,0,0,1,0,0,0,28,-0.5713,0.8208,0,278,314,307,372,314.8571,35.3196,270,LOW,0,NONE,1,0,0,NONE,YES,1,-18.6000,132.0000,132.0000,29.5000,29.5000,0.0669,2.0800,2.6900,0,0,0,0,15.9239,297,278,19,0.6100,747.8200,617.7600,130.0600,39.5200,0.0000,130.0600
300,2024-08-28,1724803200000000000,2024-08-29,1724889600000000000,SUMMER,0,0,1,0,THU,0,0,0,1,0,0,29,-0.3944,0.9190,0,337,278,314,307,309.1429,37.8701,279,HIGH,2,NONE,1,0,0,NONE,YES,1,-18.6000,132.0000,132.0000,28.3000,28.3000,0.0698,1.9700,2.4900,2,2,0,0,11.4837,307,307,-30,0.5200,764.4300,604.7900,159.6400,0.0000,15.6000,144.0400
301,2024-08-29,1724889600000000000,2024-08-30,1724976000000000000,SUMMER,0,0,1,0,FRI,0,0,0,0,1,0,30,-0.2013,0.9795,0,330,337,278,314,318.7143,34.6589,344,MEDIUM,1,NONE,1,0,0,NONE,YES,1,-18.6000,132.0000,132.0000,28.4000,28.4000,0.0728,1.9100,2.3900,1,1,0,0,-18.1398,378,330,48,0.4800,788.7000,721.9800,66.7200,91.6800,0.0000,66.7200


In [15]:
df__demand__raw.dtypes.to_frame().transpose()

,ODATE,ODATEINT,TDATE,TDATEINT,SEASON,OHE__SEASON_FALL,OHE__SEASON_SPRING,OHE__SEASON_SUMMER,OHE__SEASON_WINTER,DOW,OHE__DOW_MON,OHE__DOW_TUE,OHE__DOW_WED,OHE__DOW_THU,OHE__DOW_FRI,OHE__DOW_SAT,DOM,CE__DOM_SIN,CE__DOM_COS,HOLIDAYS,DEMAND,LAG__DEMAND_1BD,LAG__DEMAND_2BD,LAG__DEMAND_3BD,LAG__DEMAND_7BD_MEAN,LAG__DEMAND_7BD_STD,DEMAND7CD,MARKETING,ORD__MARKETING,PROMOTION,OHE__PROMOTION_NONE,OHE__PROMOTION_DISCOUNT,OHE__PROMOTION_BOGO,IMP__PROMOTION,CAF,ORD__CAF,CCI,NTI,WIN__NTI,TEMPERATURE,IMP__TEMPERATURE,PRECIPITATION,PPRC,RPRC,INT__CAF_MARKETING,INT__MARKETING_PROMOTION_NONE,INT__MARKETING_PROMOTION_DISCOUNT,INT__MARKETING_PROMOTION_BOGO,NOISE,PQTY,SQTY,INVENTORY,UGP,SR,PC,GP,CEI,CLS,NP
0,datetime64[ns],int64,datetime64[ns],int64,category,Int64,Int64,Int64,Int64,category,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Int64,category,Int64,category,Int64,Int64,Int64,category,category,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [16]:
df__demand__raw.drop(df__demand__raw.columns[df__demand__raw.columns.str.startswith(("OHE__", "ORD__", "CE__", "IMP__", "INT__", "WIN__", "LAG__", "DOM", "ODATE", "TDATE", "CCI", "NTI"))], axis=1).dtypes.to_frame().transpose()

,SEASON,DOW,HOLIDAYS,DEMAND,DEMAND7CD,MARKETING,PROMOTION,CAF,TEMPERATURE,PRECIPITATION,PPRC,RPRC,NOISE,PQTY,SQTY,INVENTORY,UGP,SR,PC,GP,CEI,CLS,NP
0,category,category,Int64,Int64,Int64,category,category,category,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64


### Business and Forecasting Metrics Calculation

Here, key business and forecasting metrics are calculated using the simulated data. These metrics provide insights into demand trends and forecast accuracy.

In [17]:
df__demand__metrics = DataFrame(columns=["KPI", "DESCRIPTION", "VALUE"])
df__demand__metrics.loc[len(df__demand__metrics), :] = ["Root Mean Squared Error", "Measures the average difference between actual demand and purchased quantity.", f"{root_mean_squared_error(df__demand__raw['DEMAND'], df__demand__raw['PQTY']):,.2f} UNITS"]
df__demand__metrics.loc[len(df__demand__metrics), :] = ["Total Cost of Inefficiency", "The combined cost of excess inventory and lost sales.", f"{(df__demand__raw['CEI'] + df__demand__raw['CLS']).sum():,.2f} EUR"]
df__demand__metrics.loc[len(df__demand__metrics), :] = ["Total Sales Revenue", "Total revenue generated from sales.", f"{df__demand__raw['SR'].sum():,.2f} EUR"]
df__demand__metrics.loc[len(df__demand__metrics), :] = ["Total Purchase Cost", "Total cost incurred from purchasing goods.", f"{df__demand__raw['PC'].sum():,.2f} EUR"]
df__demand__metrics.loc[len(df__demand__metrics), :] = ["Total Gross Profit", "Total profit before accounting for overhead costs.", f"{df__demand__raw['GP'].sum():,.2f} EUR"]
df__demand__metrics.loc[len(df__demand__metrics), :] = ["Total Net Profit", "Total profit after all expenses have been deducted.", f"{df__demand__raw['NP'].sum():,.2f} EUR"]
df__demand__metrics.loc[len(df__demand__metrics), :] = ["Total Waste Rate", "Percentage of inventory left over (waste) compared to total purchased quantity.", f"{np.abs(df__demand__raw[df__demand__raw['INVENTORY'] > 0]['INVENTORY'].sum()) / sum(df__demand__raw['PQTY']) * 100:,.2f} %"]
df__demand__metrics.loc[len(df__demand__metrics), :] = ["Total Waste Quantity", "Total number of units left as waste.", f"{np.abs(df__demand__raw[df__demand__raw['INVENTORY'] > 0]['INVENTORY'].sum())} UNITS"]
df__demand__metrics.loc[len(df__demand__metrics), :] = ["Overstock Rate", "Percentage of records where there is excess inventory.", f"{len(df__demand__raw[df__demand__raw['INVENTORY'] > 0]) / len(df__demand__raw) * 100:,.2f} %"]
df__demand__metrics.loc[len(df__demand__metrics), :] = ["Understock Rate", "Percentage of records where there is a shortage of inventory.", f"{len(df__demand__raw[df__demand__raw['INVENTORY'] < 0]) / len(df__demand__raw) * 100:,.2f} %"]
df__demand__metrics.loc[len(df__demand__metrics), :] = ["Perfect Order Rate", "Percentage of records where the inventory matched demand perfectly.", f"{len(df__demand__raw[df__demand__raw['INVENTORY'] == 0]) / len(df__demand__raw) * 100:,.2f} %"]
df__demand__metrics

,KPI,DESCRIPTION,VALUE
0,Root Mean Squared Error,Measures the average difference between actual...,46.61 UNITS
1,Total Cost of Inefficiency,The combined cost of excess inventory and lost...,"30,720.14 EUR"
2,Total Sales Revenue,Total revenue generated from sales.,"241,249.54 EUR"
3,Total Purchase Cost,Total cost incurred from purchasing goods.,"220,783.34 EUR"
4,Total Gross Profit,Total profit before accounting for overhead co...,"20,466.20 EUR"
5,Total Net Profit,Total profit after all expenses have been dedu...,"18,932.63 EUR"
6,Total Waste Rate,Percentage of inventory left over (waste) comp...,12.39 %
7,Total Waste Quantity,Total number of units left as waste.,9293 UNITS
8,Overstock Rate,Percentage of records where there is excess in...,76.57 %
9,Understock Rate,Percentage of records where there is a shortag...,23.10 %


### Validation of Simulation Parameters

This subsection involves verifying that the coefficients and parameters used in the simulation process are accurate by applying regression models to the synthetic data.

In [18]:
pipeline = make_pipeline(
    # Imputation of Missing Values
    CategoricalImputer(imputation_method="frequent", variables=["PROMOTION"]),
    # SklearnTransformerWrapper(KNNImputer(n_neighbors=4, weights="distance"), variables=["TDATEINT", "TEMPERATURE"]),
    SklearnTransformerWrapper(ForwardFillTransformer, variables=["TEMPERATURE"]),

    # Outlier Handling
    Winsorizer(capping_method="iqr", tail="right", fold=1.5, variables=["NTI"]),

    # Feature Extraction
    LagFeatures(periods=[1, 2, 3], variables=["DEMAND"]),
    WindowFeatures(window=7, periods=1, drop_na=True, functions=["mean", "std"], variables=["DEMAND"]),
    DatetimeFeatures(features_to_extract=["day_of_month", "day_of_week"], drop_original=False, variables=["TDATE"]),

    # Categorical Encoding
    CyclicalFeatures(drop_original=True, variables=["TDATE_day_of_month"]),
    SklearnTransformerWrapper(OneHotEncoder(drop=[["SUMMER"]], categories=[["WINTER", "SUMMER", "FALL", "SPRING"]], sparse_output=False), variables=["SEASON"]),
    SklearnTransformerWrapper(OneHotEncoder(drop=[["NONE"]], categories=[["NONE", "DISCOUNT", "BOGO"]], sparse_output=False), variables=["PROMOTION"]),
    SklearnTransformerWrapper(OneHotEncoder(drop=[["NO"]], categories=[["NO", "YES"]], sparse_output=False), variables=["CAF"]),
    SklearnTransformerWrapper(OneHotEncoder(drop=[[0]], categories=[[0, 1, 2, 3, 4, 5]], sparse_output=False), variables=["TDATE_day_of_week"]),
    SklearnTransformerWrapper(OrdinalEncoder(categories=[["LOW", "MEDIUM", "HIGH"]]), variables=["MARKETING"]),
    
    # Interaction Effects
    RelativeFeatures(func=["mul"], reference=["MARKETING"], variables=["CAF_YES", "PROMOTION_DISCOUNT", "PROMOTION_BOGO"]),
    
    DropFeatures(["TDATE", "DEMAND"]),
    LinearRegression()
)

In [19]:
features = ["TDATE", "DEMAND", "SEASON", "HOLIDAYS", "MARKETING", "PROMOTION", "CAF", "CCI", "NTI", "TEMPERATURE", "PRECIPITATION", "NOISE", "RPRC"]
pipeline[:-1].fit_transform(df__demand__raw[features], df__demand__raw["DEMAND"]).head(11)

,HOLIDAYS,MARKETING,CCI,NTI,TEMPERATURE,PRECIPITATION,NOISE,RPRC,DEMAND_lag_1,DEMAND_lag_2,DEMAND_lag_3,DEMAND_window_7_mean,DEMAND_window_7_std,TDATE_day_of_month_sin,TDATE_day_of_month_cos,SEASON_WINTER,SEASON_FALL,SEASON_SPRING,PROMOTION_DISCOUNT,PROMOTION_BOGO,CAF_YES,TDATE_day_of_week_1,TDATE_day_of_week_2,TDATE_day_of_week_3,TDATE_day_of_week_4,TDATE_day_of_week_5,CAF_YES_mul_MARKETING,PROMOTION_DISCOUNT_mul_MARKETING,PROMOTION_BOGO_mul_MARKETING
7,0,2.0000,-25.5000,126.0000,25.1000,0.1022,-10.6186,3.6900,217,194,154,199.8571,28.5624,0.9681,-0.2507,0.0000,1.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,2.0000,0.0000,0.0000
8,0,0.0000,-25.5000,126.0000,25.7000,0.1092,-2.2540,3.7900,267,217,194,206.8571,38.1463,0.7908,-0.6121,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9,0,0.0000,-25.5000,126.0000,24.3000,0.1128,-18.2385,3.5900,168,267,217,196.5714,37.4204,0.6514,-0.7588,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
10,0,1.0000,-25.5000,126.0000,22.8000,0.1164,-10.6900,3.4900,181,168,267,197.1429,37.1009,0.4853,-0.8743,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
11,0,0.0000,-25.5000,126.0000,23.9000,0.1201,1.2115,3.7900,158,181,168,191.2857,39.8903,0.2994,-0.9541,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000
12,0,1.0000,-25.5000,126.0000,23.4000,0.1238,1.5299,3.4900,178,158,181,194.7143,37.0842,0.1012,-0.9949,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000
13,0,0.0000,-25.5000,126.0000,22.5000,0.1275,9.1789,3.8900,223,178,158,198.8571,38.5808,-0.1012,-0.9949,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000
14,0,0.0000,-25.5000,126.0000,24.7000,0.1351,17.6108,3.4900,244,223,178,202.7143,41.9035,-0.4853,-0.8743,0.0000,1.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
15,0,0.0000,-25.5000,126.0000,21.1000,0.1389,10.6614,3.9900,203,244,223,193.5714,31.1387,-0.6514,-0.7588,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
16,0,0.0000,-25.5000,126.0000,22.4000,0.1428,19.1650,3.9900,193,203,244,197.1429,29.0828,-0.7908,-0.6121,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [20]:
pipeline = pipeline.fit(df__demand__raw[features], df__demand__raw["DEMAND"])
DataFrame([pipeline[-1].intercept_, *pipeline[-1].coef_], index=["INTERCEPT", *pipeline[-2].get_feature_names_out()]).T

,INTERCEPT,HOLIDAYS,MARKETING,CCI,NTI,TEMPERATURE,PRECIPITATION,NOISE,RPRC,DEMAND_lag_1,DEMAND_lag_2,DEMAND_lag_3,DEMAND_window_7_mean,DEMAND_window_7_std,TDATE_day_of_month_sin,TDATE_day_of_month_cos,SEASON_WINTER,SEASON_FALL,SEASON_SPRING,PROMOTION_DISCOUNT,PROMOTION_BOGO,CAF_YES,TDATE_day_of_week_1,TDATE_day_of_week_2,TDATE_day_of_week_3,TDATE_day_of_week_4,TDATE_day_of_week_5,CAF_YES_mul_MARKETING,PROMOTION_DISCOUNT_mul_MARKETING,PROMOTION_BOGO_mul_MARKETING
0,500.0000,50.0000,10.0000,10.0000,0.1000,-1.0000,-10.0000,1.0000,-20.0000,0.0500,0.0250,0.0125,0.0100,0.0050,10.0000,10.0000,-20.0000,-10.0000,-5.0000,25.0000,10.0000,-5.0000,30.0000,-10.0000,20.0000,50.0000,80.0000,4.0000,4.0000,16.0000


### Dataset Preparation for Export

This subsection covers the final steps in preparing the dataset for export, including formatting, cleaning, and ensuring that all necessary data points are included.


In [21]:
df__demand__export = df__demand__raw.copy(deep=True)
df__demand__export = df__demand__export.drop(["ODATEINT", "TDATEINT", "DOW", "DOM", "HOLIDAYS", "CCI", "NTI"], axis=1)
df__demand__export = df__demand__export.drop(df__demand__export.columns[df__demand__export.columns.str.startswith(("LAG__", "OHE__", "ORD__", "CE__", "WIN__", "IMP__", "INT__"))], axis=1)
df__demand__export.drop("NOISE", axis=1).to_csv("../datasets/demand-forecasting.csv", index=False)
df__demand__export.drop("NOISE", axis=1).to_feather("../datasets/demand-forecasting.feather")
df__demand__raw.to_feather("../datasets/demand-forecasting-raw.feather")

In [22]:
df__demand__export

,ODATE,TDATE,SEASON,DEMAND,DEMAND7CD,MARKETING,PROMOTION,CAF,TEMPERATURE,PRECIPITATION,PPRC,RPRC,NOISE,PQTY,SQTY,INVENTORY,UGP,SR,PC,GP,CEI,CLS,NP
0,2023-08-31,2023-09-01,FALL,218,276,MEDIUM,NONE,NO,28.1000,0.0759,3.1800,3.9900,7.8507,304,218,86,0.8100,869.8200,966.7200,-96.9000,273.4800,0.0000,-96.9000
1,2023-09-01,2023-09-02,FALL,240,261,MEDIUM,NONE,NO,28.0000,0.0790,3.1200,3.9900,-2.1431,287,240,47,0.8700,957.6000,895.4400,62.1600,146.6400,0.0000,62.1600
2,2023-09-02,2023-09-04,FALL,177,284,LOW,DISCOUNT,YES,26.7000,0.0854,3.1800,3.8900,0.6164,312,177,135,0.7100,688.5300,992.1600,-303.6300,429.3000,0.0000,-303.6300
3,2023-09-04,2023-09-05,FALL,199,236,LOW,DISCOUNT,YES,<NA>,0.0887,2.8000,3.3900,-16.9891,260,199,61,0.5900,674.6100,728.0000,-53.3900,170.8000,0.0000,-53.3900
4,2023-09-05,2023-09-06,FALL,154,264,LOW,NONE,YES,25.8000,0.0920,3.1800,3.9900,15.0694,290,154,136,0.8100,614.4600,922.2000,-307.7400,432.4800,0.0000,-307.7400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,2024-08-26,2024-08-27,SUMMER,314,318,MEDIUM,NONE,YES,27.7000,0.0640,2.0000,2.5900,-5.9488,350,314,36,0.5900,813.2600,700.0000,113.2600,72.0000,0.0000,113.2600
299,2024-08-27,2024-08-28,SUMMER,278,270,LOW,NONE,YES,29.5000,0.0669,2.0800,2.6900,15.9239,297,278,19,0.6100,747.8200,617.7600,130.0600,39.5200,0.0000,130.0600
300,2024-08-28,2024-08-29,SUMMER,337,279,HIGH,NONE,YES,28.3000,0.0698,1.9700,2.4900,11.4837,307,307,-30,0.5200,764.4300,604.7900,159.6400,0.0000,15.6000,144.0400
301,2024-08-29,2024-08-30,SUMMER,330,344,MEDIUM,NONE,YES,28.4000,0.0728,1.9100,2.3900,-18.1398,378,330,48,0.4800,788.7000,721.9800,66.7200,91.6800,0.0000,66.7200


In [23]:
df__demand__export.dtypes.to_frame().transpose()

,ODATE,TDATE,SEASON,DEMAND,DEMAND7CD,MARKETING,PROMOTION,CAF,TEMPERATURE,PRECIPITATION,PPRC,RPRC,NOISE,PQTY,SQTY,INVENTORY,UGP,SR,PC,GP,CEI,CLS,NP
0,datetime64[ns],datetime64[ns],category,Int64,Int64,category,category,category,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64


## Comprehensive Dataset Annotations

This section provides detailed annotations of all variables across datasets used in the project. It includes descriptions, data types, and any relevant notes on data usage and assumptions, serving as a valuable reference for understanding the dataset structure.

### Nutritional Trends Index

#### Variable Descriptions

| **Field Name** | **Data Type** | **Description**                                                                                                                                  |
|----------------|---------------|--------------------------------------------------------------------------------------------------------------------------------------------------|
| `RELEASE`      | `str`         | The release date of the Nutritional Trends Index (NTI) data. This is the date on which the NTI value is published, indicating when the data becomes available. |
| `TARGET`       | `str`         | The month and year for which the NTI is applicable. This field typically contains the abbreviated month name (e.g., "SEP" for September) and refers to the time period that the NTI value represents. |
| `TYPE`         | `str`         | The type of NTI value provided, which is always `ACTUAL`, and refers to the final NTI value observed or recorded for the target month.           |                                                                            
| `VALUE`        | `float`       | The NTI for the specified target month (`TARGET`). This index tracks shifts in consumer preferences, dietary habits, and nutritional awareness over time. |
| `VALID_FROM`   | `str`         | The start date from which the NTI value is considered valid.                                                                                     |
| `VALID_TO`     | `str`         | The end date up to which the NTI value is valid.                                                                                                 |

#### Notes on Data Usage

### Consumer Climate Index Dataset

#### Variable Descriptions

| **Field Name** | **Data Type** | **Description**                                                                                                     |
|----------------|---------------|---------------------------------------------------------------------------------------------------------------------|
| `RELEASE`      | `str`         | The release date of the Consumer Climate Index (CCI) data. This is the date on which the forecast or actual value of the CCI is published. It indicates when the data becomes available to the public or businesses for decision-making. |
| `TARGET`       | `str`         | The month and year for which the CCI is applicable. This field typically contains the abbreviated month name (e.g., "SEP" for September) and refers to the time period that the CCI value represents. |
| `TYPE`         | `str`         | The type of CCI value provided, which can be either `FORECAST` or `ACTUAL`. `FORECAST` indicates a predicted CCI value released before or at the beginning of the target month. `ACTUAL` refers to the final CCI value observed or recorded for the target month. |
| `VALUE`        | `float`       | The value of the CCI for the specified target month (`TARGET`). The CCI is a measure of consumer confidence and sentiment, with negative values indicating pessimism (reduced consumer spending and economic activity) and positive values indicating optimism (increased consumer spending and economic activity). The value is typically expressed as an index relative to a base period. |
| `VALID_FROM`   | `str`         | The start date from which the CCI value is considered valid. For `FORECAST` data, this is typically the first day of the target month (`TARGET`). For `ACTUAL` data, it is the release date (`RELEASE`). |
| `VALID_TO`     | `str`         | The end date up to which the CCI value is valid. For `FORECAST` data, this is typically the last day before the next forecast or actual value is released. For `ACTUAL` data, it is the last day of the target month (`TARGET`). This period defines the coverage of the CCI value in terms of its relevance to economic conditions. |

#### Notes on Data Usage

### German Public Holidays Dataset

#### Variable Descriptions

| **Field Name** | **Data Type** | **Description**                                                                                                     |
|----------------|---------------|---------------------------------------------------------------------------------------------------------------------|
| `DATE`         | `str`         | The date of the public holiday in Germany. This field represents the specific day on which the holiday is observed. The date is formatted as `YYYY-MM-DD`. |
| `HOLIDAY`      | `str`         | The name of the public holiday in Germany. This field contains the official name of the holiday in German, such as "Neujahr" (New Year's Day) or "Tag der Deutschen Einheit" (German Unity Day). |

#### Notes on Data Usage

### Synthetic Data for Demand Forecasting

#### Variable Descriptions

| **Field Name**  | **Data Type** | **Description**                                                                                                     |
|-----------------|---------------|---------------------------------------------------------------------------------------------------------------------|
| `ODATE`         | `str`         | The order date when all relevant information becomes available to predict the demand on the target date (`TDATE`). |
| `TDATE`         | `str`         | The target date for which the demand and other metrics are being predicted. This is the actual date for which the forecast is applicable. |
| `SEASON`        | `str`         | The season (e.g., Winter, Spring, Summer, Fall) at the target date (`TDATE`). |
| `DEMAND`        | `int`         | The actual number of units demanded or sold on the target date (`TDATE`). This is the realized demand and serves as the ground truth for evaluating the accuracy of the forecast. |
| `DEMAND7CD`     | `int`         | The actual number of units demanded or sold 7 calendar days before the target date (`TDATE`). This metric reflects the realized demand on week prior to the target date. |
| `MARKETING`     | `str`         | The level of marketing effort deployed on the target date (`TDATE`). This is typically categorized as `LOW`, `MEDIUM`, or `HIGH`, and represents the intensity of marketing campaigns, advertisements, or promotions aimed at driving sales. |
| `PROMOTION`     | `str`         | The type of promotion applied on the target date (`TDATE`), such as `NONE`, `DISCOUNT`, or `BOGO`. Promotions can have a significant impact on demand by incentivizing purchases. |
| `CAF`           | `str`         | An indicator of whether a competitor is currently running an active marketing campaign on the target date (`TDATE`). This is a binary field with values `YES` or `NO`, where `YES` indicates that a competitor has an active campaign, which could potentially impact demand. |
| `TEMPERATURE`   | `float`       | The forecasted temperature (in degrees Celsius) expected on the target date (`TDATE`), as predicted on the prediction date (`ODATE`). Weather conditions can influence consumer behavior, especially for certain seasonal products. |
| `PRECIPITATION` | `float`       | The forecasted probability of precipitation expected on the target date (`TDATE`), as predicted on the prediction date (`ODATE`). This metric represents the likelihood of rain or other forms of precipitation, which can affect foot traffic and overall demand. |
| `PPRC`          | `float`       | The purchase price per unit on the target date (`TDATE`). This is the cost at which the supermarket acquires each unit of the product from suppliers, and it directly influences the cost of goods sold (COGS). |
| `RPRC`          | `float`       | The sale price per unit on the target date (`TDATE`). This is the price at which the supermarket sells each unit to customers. It is a crucial factor in determining revenue and profitability. |
| `PQTY`          | `int`         | The quantity of units purchased on the target date (`TDATE`). This reflects the inventory replenishment made by the supermarket to meet anticipated demand. |
| `SQTY`          | `int`         | The quantity of units sold on the target date (`TDATE`). This is the actual sales volume achieved on the target date and is used to calculate revenue and inventory levels. |
| `INVENTORY`     | `int`         | The stock level on the target date (`TDATE`) after accounting for sales and purchases. A positive value indicates surplus inventory (overstocking), while a negative value indicates a shortage (understocking). Inventory management is critical for minimizing holding costs and avoiding lost sales. |
| `UGP`           | `float`       | The unit gross profit on the target date (`TDATE`), calculated as the difference between the sale price (`RPRC`) and the purchase price (`PPRC`). This metric indicates the profit margin per unit sold. |
| `SR`            | `float`       | The total sales revenue on the target date (`TDATE`), calculated as the product of the quantity sold (`SQTY`) and the sale price (`RPRC`). Sales revenue is a key indicator of the supermarkets's top-line performance. |
| `PC`            | `float`       | The total purchase cost on the target date (`TDATE`), calculated as the product of the quantity purchased (`PQTY`) and the purchase price (`PPRC`). This represents the total expenditure on acquiring inventory. |
| `GP`            | `float`       | The gross profit on the target date (`TDATE`), calculated as the difference between total sales revenue (`SR`) and total purchase cost (`PC`). Gross profit indicates the profitability before accounting for operating expenses. |
| `CEI`           | `float`       | The cost of excessive inventory on the target date (`TDATE`). This represents the holding costs incurred due to overstocking. Holding costs can include storage, insurance, spoilage, and depreciation of unsold goods. It is calculated based on the excess units in inventory that exceed expected demand. High CEI indicates inefficient inventory management and can negatively impact profitability. |
| `CLS`           | `float`       | The cost of lost sales due to understocking on the target date (`TDATE`). This cost represents the opportunity loss when demand exceeds available inventory, leading to missed sales opportunities. Understocking can result in dissatisfied customers and potential loss of market share. The cost is typically estimated based on the profit that could have been earned if sufficient inventory had been available. |
| `NP`            | `float`       | The net profit on the target date (`TDATE`), calculated as the difference between gross profit (`GP`) and the cost of lost sales (`CLS`). Net profit is a key measure of overall profitability after accounting for all relevant costs, including inefficiencies in inventory management. |
| `HOLIDAYS`      | `int`         | The number of holidays occurring after the target date (`TDATE`). Holidays are known to impact consumer behavior, and this feature helps in capturing their potential effect on demand. |

#### Coefficient Matrices for Interaction Terms

| Marketing Level | Ordinal Encoding | Effect         | Interpretation                                                                                     |
|-----------------|------------------|----------------|----------------------------------------------------------------------------------------------------|
| `LOW`           | 0                | (0 × 10) =   0 | Minimal marketing effort, so no additional demand is generated beyond the base level.              |
| `MEDIUM`        | 1                | (1 × 10) =  10 | Moderate marketing increases product visibility, leading to a slight increase in demand.           |
| `HIGH`          | 2                | (2 × 10) =  20 | High marketing significantly enhances product awareness, driving a substantial increase in demand. |

| Promotion Type | One-Hot Encoding | Effect         | Interpretation                                                                             |
|----------------|------------------|----------------|--------------------------------------------------------------------------------------------|
| `NONE`         | [1, 0, 0]        | (1 × 0) = 0    | No promotion leads to no additional demand beyond the base level.                          |
| `DISCOUNT`     | [0, 1, 0]        | (1 × 25) = 25  | Discounts are highly attractive to customers, leading to a significant increase in demand. |
| `BOGO`         | [0, 0, 1]        | (1 × 10) = 10  | The BOGO offer incentivizes purchases, moderately increasing demand.                       |

| Competitor Active Flag | Label Encoding | Effect (Units) | Interpretation     |
|------------------------|----------------|----------------|--------------------|
| `NO`                   | 0              | (0 × -5) = 0   | When there is no active competitor, sales remain at the baseline level because there is no competitive pressure affecting demand. |
| `YES`                  | 1              | (1 × -5) = -5  | When a competitor is active, sales decrease slightly due to the competition drawing away some customers, reducing demand. |

| **Marketing Level** | **Promotion Type** | **Calculation**                                                      | **Interpretation**   |
|---------------------|--------------------|----------------------------------------------------------------------|----------------------|
| `LOW`               | `NONE`             | (1 ×  0)  + (0 × 10)  + (1 × 0 ×  0) =  0 +  0 +  0 =  0 +  0 =   0  | With low marketing and no promotion, sales are at their baseline level, as there is minimal effort to attract or incentivize customers. |
| `MEDIUM`            | `NONE`             | (1 ×  0)  + (1 × 10)  + (1 × 1 ×  0) =  0 + 10 +  0 = 10 +  0 =  10  | Moderate marketing increases product visibility, leading to better sales even without any promotion. |
| `HIGH`              | `NONE`             | (1 ×  0)  + (2 × 10)  + (1 × 2 ×  0) =  0 + 20 +  0 = 20 +  0 =  20  | High marketing efforts alone significantly boost sales by maximizing product exposure, even without a promotional offer. |
| `LOW`               | `DISCOUNT`         | (1 × 25)  + (0 × 10)  + (1 × 0 ×  4) = 25 +  0 +  0 = 25 +  0 =  25  | The discount promotion increases sales somewhat, but the low level of marketing means many potential customers are not aware of the offer. |
| `MEDIUM`            | `DISCOUNT`         | (1 × 25)  + (1 × 10)  + (1 × 1 ×  4) = 25 + 10 +  4 = 35 +  4 =  39  | The combination of a discount and moderate marketing is effective in driving sales, as there is enough customer awareness and incentive to purchase. |
| `HIGH`              | `DISCOUNT`         | (1 × 25)  + (2 × 10)  + (1 × 2 ×  4) = 25 + 20 +  8 = 45 +  8 =  53  | The combination of high marketing and a discount is particularly powerful, driving the strongest sales due to the high visibility and attractive offer. |
| `LOW`               | `BOGO`             | (1 × 10)  + (0 × 10)  + (1 × 0 × 16) = 10 +  0 +  0 = 10 +  0 =  10  | The BOGO promotion has a positive effect on sales, though the low marketing effort limits the number of customers who know about it. |
| `MEDIUM`            | `BOGO`             | (1 × 10)  + (1 × 10)  + (1 × 1 × 16) = 10 + 10 + 16 = 20 + 16 =  36  | The BOGO promotion works well with moderate marketing, though its impact is slightly less than a discount, possibly due to the nature of the offer. |
| `HIGH`              | `BOGO`             | (1 × 10)  + (2 × 10)  + (1 × 2 × 16) = 10 + 20 + 32 = 30 + 32 =  62  | High marketing effectively amplifies the BOGO promotion, leading to a significant sales increase, though somewhat less than with a discount due to the different appeal of the offer. |

| **Competitor Active Flag** | **Marketing Level** | **Calculation**                                               | **Interpretation** |
|----------------------------|---------------------|---------------------------------------------------------------|--------------------|
| `NO`                       | `LOW`               | (0 × -5) + (0 × 10) + (0 × 0 × 4) =  0 +  0 + 0 =  0 + 0 =  0 | With low marketing and no active competitor, sales remain at the baseline level. There’s minimal marketing effort, and no competition to impact demand. |
| `NO`                       | `MEDIUM`            | (0 × -5) + (1 × 10) + (0 × 1 × 4) =  0 + 10 + 0 = 10 + 0 = 10 | Moderate marketing increases sales, and with no competitor, the product gains better visibility, leading to a demand increase. |
| `NO`                       | `HIGH`              | (0 × -5) + (2 × 10) + (0 × 2 × 4) =  0 + 20 + 0 = 20 + 0 = 20 | High marketing efforts significantly boost sales due to maximum product exposure, and the lack of competition ensures no negative impact on demand. |
| `YES`                      | `LOW`               | (1 × -5) + (0 × 10) + (1 × 0 × 4) = -5 +  0 + 0 = -5 + 0 = -5 | With low marketing and an active competitor, sales slightly decrease as the competitor draws away some customers, and the minimal marketing effort doesn’t counteract this loss. |
| `YES`                      | `MEDIUM`            | (1 × -5) + (1 × 10) + (1 × 1 × 4) = -5 + 10 + 4 =  5 + 4 =  9 | Moderate marketing helps offset some of the impact of the competitor, leading to a slight overall increase in sales, though not as high as it would be without competition. |
| `YES`                      | `HIGH`              | (1 × -5) + (2 × 10) + (1 × 2 × 4) = -5 + 20 + 8 = 15 + 8 = 23 | High marketing efforts boost sales, but the presence of a competitor still slightly dampens the overall impact, resulting in less demand than if there were no competition. |

#### Notes on Data Usage

Introducing interaction terms between the competitor activity flag and both marketing level and **promotion type categories provides valuable insights for strategic decision-making, as these interactions capture how competitor presence influences the effectiveness of marketing and promotional efforts, resulting in more accurate predictions and informed strategies. The synergy between marketing and promotions is evident; when combined, they amplify each other's effects, with high marketing efforts paired with strong promotions, such as discounts, leading to significant sales increases. This highlights the importance of tailoring promotions based on the level of marketing to optimize impact. However, competitor activity can diminish the effectiveness of marketing, making it crucial to consider how these external factors interact with internal efforts. While higher levels of marketing can mitigate the negative impact of competitor presence to some extent, the overall effectiveness is still reduced. Recognizing when a competitor is active allows for strategic adjustments, whether by intensifying efforts during these periods or conserving resources until the competitor's campaign ends. Understanding these dynamics enables more effective campaign planning, where timing promotions to avoid overlap with competitor activity or adjusting messaging to counteract competitor offers becomes essential. These strategies enhance marketing effectiveness and optimize promotional efforts, particularly in competitive environments, ensuring that marketing and promotion efforts are both efficient and impactful.

A sudden surge in plant-based diets due to a popular documentary could lead to overestimating salad demand. Winsorizing the Nutritional Trends Index would limit this spike's impact, ensuring it doesn't overly influence the model and result in unrealistic demand forecasts. While short-term spikes might not reflect sustained changes, overreacting to them can cause instability in predictions. By tempering these fluctuations, Winsorizing provides more reliable forecasts, aiding companies in balancing the need to respond to trends with the necessity of maintaining operational stability. Without this balance, a sudden demand spike could trigger overproduction or supply chain stress if the trend fades. Additionally, focusing on long-term trends allows Winsorizing to prevent the model from reacting too strongly to short-lived events, thus avoiding costly over-commitment of resources. Outliers can also reduce model robustness by making it overly sensitive to rare events, but Winsorizing helps maintain generalizability by limiting the impact of these extremes. However, certain scenarios may not favor Winsorizing. When the strategy involves capitalizing on spikes or when operations can rapidly scale, avoiding Winsorizing may be more advantageous, as embracing the full data range could yield better results.

#### Future Considerations

The following features have been brainstormed as potential additions to the simulated demand forecasting dataset. These features are intended to capture a wide range of economic, environmental, and behavioral factors that could influence consumer demand.

One idea is to include a **Payday Week** indicator, which could help account for potential spikes in consumer spending when individuals receive their paychecks. Another consideration is the **Consumer Price Index**, which might provide insights into inflationary pressures that could affect purchasing power. The **Local Unemployment Rate** could be another useful feature, reflecting the economic health of the area and its potential impact on consumer confidence and spending behavior. Additionally, a **Competitor Activity Index** could be developed to measure the level of promotional or pricing activities by competitors, which might influence sales.

There is also the possibility of applying Principal Component Analysis (PCA) to certain features that might share a common underlying factor, potentially representing overall consumer mood or interest. Features that could be considered for this include a **Social Media Sentiment Score**, which would gauge general sentiment toward shopping or the brand on social media platforms; a **Consumer Confidence Index**, reflecting consumers' economic outlook and willingness to spend; and a **Google Trends Score**, capturing search interest in the product category or related terms.

Several additional features are also being considered for inclusion. The **Forecasted UV Index** might help predict how likely consumers are to engage in outdoor activities, potentially influencing shopping patterns. Similarly, forecasts for the **Pollen Count** and the **Air Quality Index** could be valuable, as they could indicate factors that might keep consumers indoors, thereby affecting demand. The **Phase of the Moon** is another consideration, as it could subtly influence consumer behavior. **Daylight Hours** might also be relevant, as they could impact the time of day or the extent to which consumers are willing to shop. Other economic indicators, such as **Stock Market Performance** over the past week, could reflect broader market conditions that might influence consumer sentiment. Additionally, **Gas Prices** could be considered, as they might affect how far consumers are willing to travel for shopping, potentially impacting store visits.